In [1]:
import scanpy as sc
from lets_plot import *

import cellestial as cl

LetsPlot.setup_html()

data = sc.read("data/pbmc3k_mini.h5ad")

In [2]:
base = cl.boxplot(
    data,
    "n_genes_by_counts",
    fill="sample",
    boxplot_color="#3f3f3f",
    show_points=False,
)

In [3]:
base

In [4]:
cl.boxplots(
    data,
    ["n_genes_by_counts", "pct_counts_in_top_100_genes", "log1p_total_counts_mt","pct_counts_hb"],
    ncol=2,
    fill="sample",
    show_points=False,
    layers=[scale_y_log10()],
)